In [2]:
import json
import re

# Function to strip HTML tags
def strip_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Load the JSON data
with open('data/sampicaQuest_journals.json', 'r') as file:
    data = json.load(file)

# Clean the data
cleaned_data = []
for item in data:
    cleaned_item = {
        'id': item['id'],
        'name': item['name'],
        'entry_parsed': strip_html_tags(item['entry_parsed']),
        'calendar_year': item['calendar_year'],
        'calendar_month': item['calendar_month'],
        'calendar_day': item['calendar_day']
    }
    cleaned_data.append(cleaned_item)

# Save the cleaned data to a new JSON file
with open('data/sampicaQuest_data.json', 'w') as file:
    json.dump(cleaned_data, file, indent=4)